In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import pandas as pd
from sklearn import set_config

from sksurv.datasets import load_flchain, load_gbsg2
from sksurv.functions import StepFunction
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis
from sklearn.model_selection import GridSearchCV, KFold
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from sksurv.metrics import as_concordance_index_ipcw_scorer
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.preprocessing import OneHotEncoder, encode_categorical
from sksurv.util import Surv

import scipy.optimize as opt

set_config(display="text")  
plt.rcParams["figure.figsize"] = [7.2, 4.8]

In [2]:
def generate_marker(n_samples, m,  baseline_hazard, rnd, time_points=None):
    
    # create synthetic risk scores with uniform distribution
    X = np.array(rnd.uniform(low=-1.0, high = 1.0, size=(n_samples, m)))
    hazard_ratio = np.expand_dims(np.array(rnd.uniform(low = 0, high = 0.1, size=m)), axis=0)
    coef = np.log(hazard_ratio)

    # create linear model
    logits = np.dot(X, coef.T)

    # draw actual survival times from exponential distribution,
    # refer to Bender et al. (2005), https://doi.org/10.1002/sim.2059
    u = rnd.uniform(size=n_samples)
    risk_scores = np.squeeze(logits)
    time_event = -np.log(u) / (baseline_hazard * np.exp(risk_scores))

    # calculate actual concordance (with no censoring)
    actual = concordance_index_censored(np.ones(n_samples, dtype=bool), time_event, risk_scores)

    # calculate baseline AUC (with no censoring)
    y_uncensored = np.array([(True, t) for t in time_event], 
                  dtype=[('event', bool), ('time', float)])
    
    baseline_auc, baseline_mean_auc = cumulative_dynamic_auc(
        y_uncensored, y_uncensored, risk_scores, time_points
    )

    return X, time_event, actual[0], hazard_ratio, risk_scores, baseline_auc, baseline_mean_auc, time_points

In [3]:
def generate_survival_data(n_samples, m, baseline_hazard, percentage_cens, rnd, time_points=None, retry=True, evaluate_auc=True):
                 
    X, time_event, actual_c, hazard_ratio, risk_scores, baseline_auc, baseline_mean_auc, eval_time_points = generate_marker(
        n_samples, m, baseline_hazard, rnd, time_points)

    def get_observed_time(x): # this censors certain time events
        rnd_cens = rnd
        # draw censoring times
        time_censor = rnd_cens.uniform(high=x, size=n_samples)
        event = time_event < time_censor
        time = np.where(event, time_event, time_censor)
        return event, time # returns bool array of if event occured or not/censored, and the time it occured/ was censored

    def censoring_amount(x): # finds optimal time event for censoring to be as close to desired censored amount
        event, _ = get_observed_time(x)
        cens = 1.0 - event.sum() / event.shape[0]
        return (cens - percentage_cens) ** 2

    # search for upper limit to obtain the desired censoring amount
    res = opt.minimize_scalar(censoring_amount, method="bounded", bounds=(0, time_event.max()))

    if ( np.abs(res.fun) > 2.0/n_samples and retry): # check for convergence
        return generate_survival_data(n_samples, m, baseline_hazard, percentage_cens, rnd=rnd, retry=False, time_points=time_points)
    elif (np.abs(res.fun) > 2.0/n_samples and not retry):
        converged = False
    else:
        converged = True

    # compute observed time
    event, time = get_observed_time(res.x) # now that we have the optimal time event, we use that to get all the events and times

    # tau limits the data we look at. Here we are keeping events where time is < latest observed event time. (This decrease biases)
    tau = time[event].max()
    y = Surv.from_arrays(event=event, time=time)
    mask = time < tau
    X_test = X[mask]
    y_test = y[mask]

    return X_test, y_test, y, actual_c, converged, hazard_ratio, risk_scores[mask], baseline_mean_auc, eval_time_points


In [4]:
def simulation(n_samples, m, n_repeats=100, time_points=10, use_gridsearch=True):

    rnd = np.random.RandomState(42) 
    
    measures = (
        "censoring",
        "Actual C",
        "Harrel's C",
        "Uno's C",
        "Baseline AUC",
        "AUC",
        "Brier",
    )
    results = {
        "rsf": {"mean": [], "std": [], "censoring": []},    
    }
    
    # hyperparameter tuning
    param_grid = {
        'n_estimators': [50, 100, 200],
        'min_samples_split': [5, 10, 20],
        'min_samples_leaf': [5, 10, 15],
        'max_features': ['sqrt', 'log2', None],
        'max_depth': [3, 5, 7, None]
    }
    
    best_params_per_censoring = {}
    
    # iterate over different amount of censoring
    for cens in [0, 0.25, 0.5]:
        data = {
            "rsf": {measure: [] for measure in measures},
        }

        # perform grid search once per censoring level (not per repeat)
        if use_gridsearch:
            print(f"Running GridSearch for censoring level {cens}")
            
            # Generate a representative dataset for grid search
            X_grid, y_grid, _, _, _, _, _, _, _ = generate_survival_data(
                n_samples, m, baseline_hazard=0.1, percentage_cens=cens, rnd=rnd, time_points=time_points
            )

            base_model = RandomSurvivalForest(random_state=rnd, n_jobs=1)
            scorer = as_concordance_index_ipcw_scorer(base_model)

                     
            cv = KFold(n_splits=3, shuffle=True, random_state=rnd)
            grid_search = GridSearchCV(
                estimator=scorer,  
                param_grid={'estimator__' + k: v for k, v in param_grid.items()},  # Prefix with 'estimator__'
                cv=cv,
                n_jobs=-1,
                verbose=1,
                error_score='raise'
            )
            
            try:
                grid_search.fit(X_grid, y_grid)
                
                best_params = {k.replace('estimator__', ''): v 
                             for k, v in grid_search.best_params_.items()}
                best_params_per_censoring[cens] = best_params
                
                print(f"Best parameters for censoring {cens}: {best_params}")
                print(f"Best CV score: {grid_search.best_score_:.3f}")
                
            except Exception as e:
                print(f"GridSearch failed for censoring {cens}: {str(e)}")
                # default parameters
                best_params_per_censoring[cens] = {
                    'n_estimators': 100,
                    'min_samples_split': 10,
                    'min_samples_leaf': 15,
                    'max_features': 'sqrt',
                    'max_depth': None
                }
        else:
            # default parameters if grid search is disabled
            best_params_per_censoring[cens] = {
                'n_estimators': 100,
                'min_samples_split': 10,
                'min_samples_leaf': 15,
                'max_features': 'sqrt',
                'max_depth': None
            }

        # Now perform repeated simulations with the best parameters
        for repeat_idx in range(n_repeats):
            
            X_test, y_test, y_train, actual_c, converged, hazard_ratio, true_risk_scores, baseline_mean_auc, eval_time_points = generate_survival_data(
                n_samples, m, baseline_hazard=0.1, percentage_cens=cens, rnd=rnd, time_points=time_points
            )
        
            if not converged:
                continue  # Skip this repeat if convergence failed

            for model_type in ["rsf"]:
                # Use the best parameters found for this censoring level
                best_params = best_params_per_censoring[cens]
                
                # Create model with best parameters
                model = RandomSurvivalForest(
                    random_state=rnd,
                    n_jobs=-1,
                    **best_params
                )
                
                try:
                    model.fit(X_test, y_test)
                except Exception as e:
                    print(f"Model fitting failed for repeat {repeat_idx+1}: {str(e)}")
                    continue

                # predict risk scores 
                risk_scores = model.predict(X_test)
                
                # Generate time points for evaluation
                times = np.linspace(y_test["time"].min() + 0.001, y_test["time"].max() - 0.001, time_points)

                # Compute cumulative dynamic AUC
                try:
                    aucs, _ = cumulative_dynamic_auc(y_train, y_test, risk_scores, times)
                    mean_auc = np.nanmean(aucs)
                except Exception as e:
                    print(f"AUC calculation failed: {e}")
                    mean_auc = np.nan

                # Brier Score - requires survival function predictions
                try:
                    pred_func = model.predict_survival_function(X_test)
                    preds = np.asarray([[fn(t) for t in times] for fn in pred_func])
                    brier = integrated_brier_score(y_train, y_test, preds, times)
                except Exception as e:
                    print(f"Brier score calculation failed: {e}")
                    brier = np.nan

                # Estimate c-indices
                try:
                    c_actual = actual_c
                    c_harrell = concordance_index_censored(y_test["event"], y_test["time"], risk_scores)
                    c_uno = concordance_index_ipcw(y_train, y_test, risk_scores)
                    
                    # Calculate actual censoring percentage
                    censoring_pct = (1 - y_test["event"].sum() / y_test.shape[0]) * 100.0
                    
                    # Save results
                    data[model_type]["censoring"].append(censoring_pct)
                    data[model_type]["Actual C"].append(c_actual)
                    data[model_type]["Harrel's C"].append(c_harrell[0])
                    data[model_type]["Uno's C"].append(c_uno[0])
                    data[model_type]["Baseline AUC"].append(baseline_mean_auc)
                    data[model_type]["AUC"].append(mean_auc)
                    data[model_type]["Brier"].append(brier)
                    
                except Exception as e:
                    print(f"Metric calculation failed: {e}")
                    continue

        # Aggregate results for this censoring level
        for model_type in ["rsf"]:
            data_mean = {key: [np.mean(value)] for key, value in data[model_type].items()}
            data_std = {key: [np.std(value, ddof=1)] for key, value in data[model_type].items()}

            results[model_type]["mean"].append(pd.DataFrame(data_mean))
            results[model_type]["std"].append(pd.DataFrame(data_std))

    return results

In [5]:
def plot_results(results, model_type="rsf", **kwargs):
    
    # Concatenate DataFrames from the list of results
    data_mean = pd.concat(results[model_type]["mean"])
    data_std = pd.concat(results[model_type]["std"])
    
    # Create index based on censoring values
    index = pd.Index(data_mean["censoring"].round(3), name="mean percentage censoring")
    for df in (data_mean, data_std):
        df.drop("censoring", axis=1, inplace=True)
        df.index = index
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 6), sharex=True)
    
    # Concordance indexes plot
    cindex_columns = ["Actual C", "Harrel's C", "Uno's C"]
    data_mean_cindex = data_mean[cindex_columns]
    data_std_cindex = data_std[cindex_columns]
    
    # Plot the bar chart for concordance indexes
    data_mean_cindex.plot.bar(
        yerr=data_std_cindex,
        ax=axes[0],
        width=0.7,
        linewidth=0.5,
        capsize=4,
        **kwargs
    )
    axes[0].set_ylabel("Concordance")
    axes[0].set_title("Concordance Index Errors")
    axes[0].yaxis.grid(True, linestyle='--', alpha=0.7)
    axes[0].set_ylim(0, 1)  # Set y-axis limit to match the image
    axes[0].axhline(0.0, color="gray", linewidth=0.8)
    
    # AUC comparison plot (Baseline vs Model)
    auc_columns = ["Baseline AUC", "AUC"]
    data_mean_auc = data_mean[auc_columns]
    data_std_auc = data_std[auc_columns]
    
    # Plot the bar chart for AUC comparison
    data_mean_auc.plot.bar(
        yerr=data_std_auc,
        ax=axes[1],
        width=0.7,
        linewidth=0.5,
        capsize=4,
        **kwargs
    )
    axes[1].set_ylabel("AUC Score")
    axes[1].set_title("AUC Comparison: Baseline vs Model")
    axes[1].yaxis.grid(True, linestyle='--', alpha=0.7)
    axes[1].set_ylim(0, 1.0)
    
    # Brier score plot
    brier_column = ["Brier"]
    data_mean_brier = data_mean[brier_column]
    data_std_brier = data_std[brier_column]
    
    # Plot the bar chart for Brier scores
    data_mean_brier.plot.bar(
        yerr=data_std_brier,
        ax=axes[2],
        width=0.7,
        linewidth=0.5,
        capsize=4,
        **kwargs
    )
    axes[2].set_ylabel("Brier Score")
    axes[2].set_title("Integrated Brier Score")
    axes[2].yaxis.grid(True, linestyle='--', alpha=0.7)
    axes[2].set_ylim(0, 0.5)  # Brier scores are typically lower, adjust as needed
    
    # x-axis label on the bottom plot
    for ax in axes:
        ax.set_xlabel("Mean Percentage Censoring")
    
    # Remove top and right spines for all axes
    for ax in axes:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.tick_params(axis='x', labelrotation=90)
    
    plt.tight_layout()
    return fig, axes

In [6]:
import warnings
warnings.filterwarnings('ignore')
results = simulation(n_samples=1000, m=3, n_repeats=5, time_points=10)

Running GridSearch for censoring level 0
Fitting 3 folds for each of 324 candidates, totalling 972 fits
GridSearch failed for censoring 0: censoring survival function is zero at one or more time points
Running GridSearch for censoring level 0.25
Fitting 3 folds for each of 324 candidates, totalling 972 fits
Best parameters for censoring 0.25: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 200}
Best CV score: 0.907
Running GridSearch for censoring level 0.5
Fitting 3 folds for each of 324 candidates, totalling 972 fits
GridSearch failed for censoring 0.5: censoring survival function is zero at one or more time points
